In [2]:
# test.py
import pandas as pd
from News_Crawler import crawling_news

df = pd.DataFrame(crawling_news.get_req())
df

/home/kar/Projects/News_Summary/venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:548: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


,title,points,link
0,Novops - 크로스 플랫폼 Secret & Config 관리자 (github.c...,6,https://github.com/PierreBeucher/novops
1,"Lady Deirdre - 프로그래밍 언어 컴파일러, 인터프리터 및 소스 코드 분석...",9,https://github.com/Eliah-Lakhin/lady-deirdre
2,GN⁺: Konty – 현대 앱을 위한 lo-fi 와이어프레임 도구. Balsami...,21,https://konty.app/
3,GN⁺: 파인트(Pint) 사이즈 스타트업이 미국 경제를 재편하고 있다 (wsj.com),4,https://www.wsj.com/business/entrepreneurship/...
4,GN⁺: 도난당한 아이폰은 iOS 18 이후부터 더욱 쓸모없어집니다 (macobse...,2,https://www.macobserver.com/ios/stolen-iphones...
5,"GN⁺: Just for Fun. No, Really. - 재미삼아 개발했어요. 진...",49,https://justforfunnoreally.dev
6,GN⁺: 왜 Haskell인가? (gtf.io),16,https://www.gtf.io/musings/why-haskell
7,Reader-LM : HTML을 마크다운으로 변환하는 작은 언어 모델 (jina.ai),13,https://jina.ai/news/reader-lm-small-language-...
8,GN⁺: B-트리와 데이터베이스 인덱스 (planetscale.com),37,https://planetscale.com/blog/btrees-and-databa...
9,'취미' 앱이 새로운 소셜 네트워크가 될까? (theguardian.com),31,https://www.theguardian.com/technology/article...


In [3]:
df['link']

0               https://github.com/PierreBeucher/novops
1          https://github.com/Eliah-Lakhin/lady-deirdre
2                                    https://konty.app/
3     https://www.wsj.com/business/entrepreneurship/...
4     https://www.macobserver.com/ios/stolen-iphones...
5                        https://justforfunnoreally.dev
6                https://www.gtf.io/musings/why-haskell
7     https://jina.ai/news/reader-lm-small-language-...
8     https://planetscale.com/blog/btrees-and-databa...
9     https://www.theguardian.com/technology/article...
10    https://twitter.com/SmokeAwayyy/status/1834641...
11     https://simonwillison.net/2024/Sep/12/openai-o1/
12    https://softwaredoug.com/blog/2024/09/07/your-...
13                  https://news.hada.io/topic?id=16731
14    https://calebporzio.com/i-just-cracked-1-milli...
15    https://thenewstack.io/one-company-rethinks-di...
16                https://yaak.app/blog/now-open-source
17                  https://news.hada.io/topic?i

In [4]:
import requests
from bs4 import BeautifulSoup

contents = []

for i in range(len(df['link'])):
    res = requests.get(df['link'][i], auth=("user", "pass")) 

    if res.status_code == 200:        
        soup = BeautifulSoup(res.text, 'lxml')
        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()  # rip it out
        
        # get text
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        contents.append(text)
    elif res.status_code == 403:
        contents.append('e403')

    else:
        contents.append('e404')

In [6]:
pd.concat([df, pd.Series(contents, name='content')], axis=1)

,title,points,link,content
0,Novops - 크로스 플랫폼 Secret & Config 관리자 (github.c...,6,https://github.com/PierreBeucher/novops,GitHub - PierreBeucher/novops: Cross-platform ...
1,"Lady Deirdre - 프로그래밍 언어 컴파일러, 인터프리터 및 소스 코드 분석...",9,https://github.com/Eliah-Lakhin/lady-deirdre,GitHub - Eliah-Lakhin/lady-deirdre: Compiler f...
2,GN⁺: Konty – 현대 앱을 위한 lo-fi 와이어프레임 도구. Balsami...,21,https://konty.app/,Konty | Sketch your app ideas without stress\n...
3,GN⁺: 파인트(Pint) 사이즈 스타트업이 미국 경제를 재편하고 있다 (wsj.com),4,https://www.wsj.com/business/entrepreneurship/...,e403
4,GN⁺: 도난당한 아이폰은 iOS 18 이후부터 더욱 쓸모없어집니다 (macobse...,2,https://www.macobserver.com/ios/stolen-iphones...,Stolen iPhones Will Be Even More Useless From ...
5,"GN⁺: Just for Fun. No, Really. - 재미삼아 개발했어요. 진...",49,https://justforfunnoreally.dev,"Just for Fun. No, Really.\nJust for Fun. No, R..."
6,GN⁺: 왜 Haskell인가? (gtf.io),16,https://www.gtf.io/musings/why-haskell,GTF :: Why Haskell?~gtfoh the places you'll go...
7,Reader-LM : HTML을 마크다운으로 변환하는 작은 언어 모델 (jina.ai),13,https://jina.ai/news/reader-lm-small-language-...,Reader-LM: Small Language Models for Cleaning ...
8,GN⁺: B-트리와 데이터베이스 인덱스 (planetscale.com),37,https://planetscale.com/blog/btrees-and-databa...,B-trees and database indexesSkip to contentDoc...
9,'취미' 앱이 새로운 소셜 네트워크가 될까? (theguardian.com),31,https://www.theguardian.com/technology/article...,"Goodbye Tinder, hello Strava: have ‘hobby’ app..."
